https://qiita.com/hitokun-s/items/fdbe0a3b103801274aa8
自然言語によるタスク指示で、所定の処理に所定のAPIを使うよう誘導する

In [3]:
import json
import requests

def search_book(message_json: str):
    """return book info for given ISBN
    :param message_json: JSON string like {"isbn": "<ISBN>"}
    """

    try:
        isbn = json.loads(message_json)["isbn"]
    except ValueError as e:
        return "invalid input format. format should be JSON string having 'isbn' key"

    url = f"https://api.openbd.jp/v1/get?isbn={isbn}&pretty"
    response = requests.get(url)

    if response.status_code != 200:
        return "Failed to request to OpenBD API."

    book_data = response.json()
    if len(book_data) == 0:
        return "No book info found for given ISBN."

    return book_data[0]["summary"] # dict having "author", "title", "publisher"


In [5]:
'''
from langchain.agents import load_tools
tool_names = ["python_repl"]
tools = load_tools(tool_names)
print(tools)
'''
from langchain.agents import load_tools
from langchain.agents import Tool
from langchain.tools.base import BaseTool
from langchain.llms import LlamaCpp
from langchain.agents import initialize_agent
from langchain.agents import AgentType

# from langchain.tools.base import BaseTool
class HelloTool(BaseTool):
    """Tool that generates a personalized hello message."""
    name="get book information"
    func=search_book
    description="""helps to get book information(title, author, publisher) from its ISBN number.
    Before using this API, you must get ISBN of the book.
    Input should be json in the following format: `{{"isbn": "<ISBN number>"}}`
    Output is a JSON in the following format: `{{"title": "<title>", "author": "<author>"}}`
    """
    def _run(self, name: str = None) -> str:
        return f"Hello {name}!"
    async def _arun(self, name: str = None) -> str:
        """Use the HelloTool asynchronously."""
        return self._run(name)
    
tools = [HelloTool()]

mytool = Tool(
        name="get book information",
        func=search_book,
        description="""helps to get book information(title, author, publisher) from its ISBN number.
    Before using this API, you must get ISBN of the book.
    Input should be json in the following format: `{{"isbn": "<ISBN number>"}}`
    Output is a JSON in the following format: `{{"title": "<title>", "author": "<author>"}}`
    """
)

tools.append(mytool)

# OpenAIのLLMを初期化
# llm = OpenAI(temperature=0, verbose=True)
llm = LlamaCpp(
    model_path="llama.cpp/models/rinna-youri-7b-chat-q4_K_M.gguf",
    # model_path=r"llama.cpp/cpp\models\llama-2-7b-chat.Q4_K_M.gguf",
    input={
        "max_tokens": 32,
        "stop": ["System:", "User:", "Assistant:", "\n"],
    },
    verbose=True,
    temperature=1,
    n_ctx=2048
)

# 実験のため、google-search toolとカスタムツールを併用
# tools = load_tools(["google-search"], llm=llm)

# mrkl = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)
executor = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)




c:\Users\onowa\miniconda3\envs\vir_env\lib\site-packages\langchain\utils\utils.py:159: UserWarning: WARNING! input is not default parameter.
                input was transferred to model_kwargs.
                Please confirm that input is what you intended.
  warnings.warn(
AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [6]:
question = "千葉ルーという本の詳細を教えてください。著者や出版社など。"
executor.run(question)



> Entering new AgentExecutor chain...
  これに対しては、次のようなアプローチが適用できます。
Action:  「千葉ルー」と検索エンジンを使って調べてください。
Action Input:  おそらく、出版社のリストが見つかるはずです。
Observation: 「千葉ルー」と検索エンジンを使って調べてください。 is not a valid tool, try one of [get book information].
Thought:

Llama.generate: prefix-match hit


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `  次に、この本がどのような本であるかについて話し合おう。`